# Reading and merging data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data from 09/2020 to 12/2020 for high volume fhv

fhvhv_0920_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-09.feather')
fhvhv_1020_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-10.feather')
fhvhv_1120_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-11.feather')
fhvhv_1220_feather = pd.read_feather('/Volumes/E/data/feather/fhvhv_tripdata_2020-12.feather')


In [3]:
# Merge the data from Sep to Dec

frames = [fhvhv_0920_feather, fhvhv_1020_feather, fhvhv_1120_feather, fhvhv_1220_feather]
fhvhv_2020_df = pd.concat(frames, keys=["Sep", "Oct", "Nov", "Dec"])
fhvhv_2020_df.head()

hvfhs_license_num dispatching_base_num      pickup_datetime  \
Sep 0            HV0003               B02764  2020-09-01 00:14:27   
    1            HV0003               B02764  2020-09-01 00:31:36   
    2            HV0003               B02682  2020-09-01 00:19:08   
    3            HV0003               B02764  2020-09-01 00:32:09   
    4            HV0003               B02878  2020-09-01 00:06:58   

          dropoff_datetime  PULocationID  DOLocationID  SR_Flag  
Sep 0  2020-09-01 00:20:02           170           230      NaN  
    1  2020-09-01 00:52:06           163           265      NaN  
    2  2020-09-01 00:23:52           223           223      NaN  
    3  2020-09-01 01:08:58             4           265      NaN  
    4  2020-09-01 00:24:47           244           254      NaN

In [4]:
# Data that records covid cases by day in nyc

covid_df = pd.read_csv('/Volumes/E/data/cases-by-day.csv')
covid_df.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BK_CASE_COUNT,...,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,0


# Data cleaning and preprocessing

High Volume FHV data

In [5]:
fhvhv_2020_df.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [6]:
fhvhv_2020_df.drop(columns=['dispatching_base_num'], inplace=True)

In [7]:
# Impute missing values for share rides 
fhvhv_2020_df['SR_Flag'].fillna(0, inplace=True)
fhvhv_2020_df.dropna(0, inplace = True)
print("Done filling missing values")

# Convert share rides to categorical data
fhvhv_2020_df['SR_Flag'] = fhvhv_2020_df['SR_Flag'].astype('bool')

# Check if any rows have missing values, if not, proceed
fhvhv_2020_df.isnull().sum().sort_values(ascending=False)

Done filling missing values


hvfhs_license_num    0
pickup_datetime      0
dropoff_datetime     0
PULocationID         0
DOLocationID         0
SR_Flag              0
dtype: int64

In [8]:
# Add a new column which comprises the duration of each trip in minutes
# Codes are modified based on:
# https://stackoverflow.com/questions/51938140/how-to-get-the-time-duration-from-two-date-time-columns-of-pandas-dataframe
# Might take up to 10 minutes to load...

covert_to_min = 60

fhvhv_2020_df['pickup_datetime'] = pd.to_datetime(fhvhv_2020_df['pickup_datetime'])
fhvhv_2020_df['dropoff_datetime'] = pd.to_datetime(fhvhv_2020_df['dropoff_datetime'])
fhvhv_2020_df['trip_duration_min'] = (fhvhv_2020_df['dropoff_datetime'] - fhvhv_2020_df['pickup_datetime']).apply(
    lambda x: x.seconds/(covert_to_min))
fhvhv_2020_df['trip_duration_min'] = fhvhv_2020_df['trip_duration_min'].round(2)

fhvhv_2020_df.head()

hvfhs_license_num     pickup_datetime    dropoff_datetime  PULocationID  \
Sep 0            HV0003 2020-09-01 00:14:27 2020-09-01 00:20:02           170   
    1            HV0003 2020-09-01 00:31:36 2020-09-01 00:52:06           163   
    2            HV0003 2020-09-01 00:19:08 2020-09-01 00:23:52           223   
    3            HV0003 2020-09-01 00:32:09 2020-09-01 01:08:58             4   
    4            HV0003 2020-09-01 00:06:58 2020-09-01 00:24:47           244   

       DOLocationID  SR_Flag  trip_duration_min  
Sep 0           230    False               5.58  
    1           265    False              20.50  
    2           223    False               4.73  
    3           265    False              36.82  
    4           254    False              17.82

In [9]:
# Remove outliers: minimum and maximum values of trip duration

outlier_min = fhvhv_2020_df['trip_duration_min'].min()
print('Minimum trip duration: ',outlier_min)
outlier_max = fhvhv_2020_df['trip_duration_min'].max()
print('Maximum trip duration: ',outlier_max)

fhvhv_2020_df = fhvhv_2020_df[(fhvhv_2020_df.trip_duration_min != outlier_min)
                              & (fhvhv_2020_df.trip_duration_min != outlier_max)]    
print('Outliers removed!')

Minimum trip duration:  0.0
Maximum trip duration:  1439.88
Outliers removed!


In [10]:
# Generate a new column pick_day indicating the day of pickup

fhvhv_2020_df['pickup_day'] = pd.DatetimeIndex(fhvhv_2020_df['pickup_datetime']).weekday


COVID-19 data

In [11]:
# Check data type
covid_df.dtypes

date_of_interest              object
CASE_COUNT                     int64
PROBABLE_CASE_COUNT            int64
CASE_COUNT_7DAY_AVG            int64
ALL_CASE_COUNT_7DAY_AVG        int64
BX_CASE_COUNT                  int64
BX_PROBABLE_CASE_COUNT         int64
BX_CASE_COUNT_7DAY_AVG         int64
BX_ALL_CASE_COUNT_7DAY_AVG     int64
BK_CASE_COUNT                  int64
BK_PROBABLE_CASE_COUNT         int64
BK_CASE_COUNT_7DAY_AVG         int64
BK_ALL_CASE_COUNT_7DAY_AVG     int64
MN_CASE_COUNT                  int64
MN_PROBABLE_CASE_COUNT         int64
MN_CASE_COUNT_7DAY_AVG         int64
MN_ALL_CASE_COUNT_7DAY_AVG     int64
QN_CASE_COUNT                  int64
QN_PROBABLE_CASE_COUNT         int64
QN_CASE_COUNT_7DAY_AVG         int64
QN_ALL_CASE_COUNT_7DAY_AVG     int64
SI_CASE_COUNT                  int64
SI_PROBABLE_CASE_COUNT         int64
SI_CASE_COUNT_7DAY_AVG         int64
SI_ALL_CASE_COUNT_7DAY_AVG     int64
INCOMPLETE                     int64
dtype: object

In [12]:
# Drop this column as it is irrelevant to analysis
covid_df.drop(['INCOMPLETE'], axis=1, inplace=True)

In [13]:
# Only keep the data within the time period of interest

start_date = '2020-09-01'
end_date = '2020-12-31'

covid_df['date_of_interest'] = pd.to_datetime(covid_df['date_of_interest'])
time_period = (covid_df['date_of_interest'] >= start_date) & (covid_df['date_of_interest'] <= end_date)
covid_df = covid_df.loc[time_period].reset_index(drop=True)

# Descriptive Statistics

High Volume FHV data

In [14]:
# Check dimension of the data
fhvhv_2020_df.shape

(48556503, 8)

In [15]:
pd.options.display.float_format = '{:,.2f}'.format
fhvhv_2020_df.describe()

,PULocationID,DOLocationID,trip_duration_min,pickup_day
count,"48,556,503.00","48,556,503.00","48,556,503.00","48,556,503.00"
mean,135.50,139.01,17.41,3.13
std,76.77,78.65,20.64,1.95
min,1.00,1.00,0.02,0.00
25%,70.00,72.00,9.32,1.00
50%,137.00,140.00,14.35,3.00
75%,210.00,215.00,21.87,5.00
max,265.00,265.00,"1,439.82",6.00


In [16]:
# Count the number of trips according to HVFHS business
# HV0002: Juno
# HV0003: Uber
# HV0004: Via
# HV0005: Lyft

print("Number of trips according to HVFHS business from 09/2020 to 12/2020:")
fhvhv_2020_df['hvfhs_license_num'].value_counts()

Number of trips according to HVFHS business from 09/2020 to 12/2020:


HV0003    35456920
HV0005    12551974
HV0004      547609
Name: hvfhs_license_num, dtype: int64

COVID-19 data

In [17]:
covid_df.describe()

,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
count,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,...,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00
mean,"1,384.39",380.81,"1,283.34","1,640.88",233.26,66.03,214.01,276.02,417.62,105.80,...,193.16,251.85,385.06,111.18,354.36,457.93,143.03,35.98,132.16,166.16
std,"1,250.63",343.92,"1,067.22","1,386.57",227.90,62.47,185.77,243.09,352.26,92.24,...,150.95,201.94,371.58,107.68,316.84,416.02,137.44,32.00,119.20,148.07
min,156.00,11.00,236.00,259.00,18.00,1.00,37.00,42.00,37.00,3.00,...,37.00,41.00,28.00,3.00,58.00,62.00,13.00,0.00,18.00,19.00
25%,417.50,65.25,470.25,539.00,63.25,9.25,59.25,67.00,168.00,28.25,...,64.50,71.50,104.50,16.00,116.75,135.25,31.25,4.00,31.00,36.00
50%,774.00,234.00,623.00,826.00,123.50,39.50,105.00,137.50,245.00,71.50,...,101.50,142.50,191.00,61.50,155.50,204.50,83.00,31.00,65.50,92.00
75%,"2,150.25",639.25,"2,094.00","2,760.25",355.75,112.75,357.50,476.50,593.25,177.75,...,325.00,445.25,603.00,188.50,564.00,749.75,239.50,63.00,260.50,323.50
max,"5,385.00","1,196.00","3,761.00","4,682.00","1,019.00",212.00,687.00,847.00,"1,600.00",346.00,...,467.00,588.00,"1,602.00",390.00,"1,125.00","1,424.00",584.00,107.00,400.00,481.00


# Data aggregation

In [18]:
# Prepare to aggregate fhvhv taxi data and covid data
# Takes about 5 min to load

# Keep attributes we are interested in
fhvhv_by_day_df = fhvhv_2020_df[['pickup_datetime','SR_Flag','trip_duration_min']]

# To match the datetime template in covid data
fhvhv_by_day_df['pickup_date'] = fhvhv_2020_df['pickup_datetime'].dt.strftime('%Y-%m-%d')

fhvhv_by_day_df.head()

pickup_datetime  SR_Flag  trip_duration_min pickup_date
Sep 0 2020-09-01 00:14:27    False               5.58  2020-09-01
    1 2020-09-01 00:31:36    False              20.50  2020-09-01
    2 2020-09-01 00:19:08    False               4.73  2020-09-01
    3 2020-09-01 00:32:09    False              36.82  2020-09-01
    4 2020-09-01 00:06:58    False              17.82  2020-09-01

In [19]:
# Aggregating all attributes by date
# Codes are modified based on:
# https://stackoverflow.com/questions/45752601/how-to-do-a-conditional-count-after-groupby-on-a-pandas-dataframe


reduced_df = fhvhv_by_day_df.groupby('pickup_date')['SR_Flag'].apply(
    lambda x: (x==True).sum()).reset_index(name='num_of_SR')
reduced_df['trip_count'] = fhvhv_by_day_df.groupby('pickup_date')['pickup_date'].agg('count').reset_index(
    name='trip').trip
reduced_df['avg_trip_duration'] = fhvhv_by_day_df.groupby('pickup_date')['trip_duration_min'].agg('mean').reset_index(
    name='duration').duration
reduced_df.head()

,pickup_date,num_of_SR,trip_count,avg_trip_duration
0,2020-09-01,34,344879,17.83
1,2020-09-02,42,357969,17.72
2,2020-09-03,26,391021,18.26
3,2020-09-04,6,425570,19.12
4,2020-09-05,0,434656,17.15


In [20]:
# Combine the fhvhv data with covid data

fhvhv_covid_df = pd.merge(reduced_df,covid_df,left_index=True,right_index=True)
fhvhv_covid_df = fhvhv_covid_df.drop(columns=['date_of_interest'])
fhvhv_covid_df.describe()

,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
count,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,...,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00,122.00
mean,37.43,"398,004.12",17.34,"1,384.39",380.81,"1,283.34","1,640.88",233.26,66.03,214.01,...,193.16,251.85,385.06,111.18,354.36,457.93,143.03,35.98,132.16,166.16
std,32.30,"58,989.20",2.02,"1,250.63",343.92,"1,067.22","1,386.57",227.90,62.47,185.77,...,150.95,201.94,371.58,107.68,316.84,416.02,137.44,32.00,119.20,148.07
min,0.00,"285,895.00",15.06,156.00,11.00,236.00,259.00,18.00,1.00,37.00,...,37.00,41.00,28.00,3.00,58.00,62.00,13.00,0.00,18.00,19.00
25%,0.00,"357,020.75",16.41,417.50,65.25,470.25,539.00,63.25,9.25,59.25,...,64.50,71.50,104.50,16.00,116.75,135.25,31.25,4.00,31.00,36.00
50%,41.00,"380,175.00",17.33,774.00,234.00,623.00,826.00,123.50,39.50,105.00,...,101.50,142.50,191.00,61.50,155.50,204.50,83.00,31.00,65.50,92.00
75%,68.00,"430,042.50",17.86,"2,150.25",639.25,"2,094.00","2,760.25",355.75,112.75,357.50,...,325.00,445.25,603.00,188.50,564.00,749.75,239.50,63.00,260.50,323.50
max,96.00,"596,817.00",36.77,"5,385.00","1,196.00","3,761.00","4,682.00","1,019.00",212.00,687.00,...,467.00,588.00,"1,602.00",390.00,"1,125.00","1,424.00",584.00,107.00,400.00,481.00


# Saving preprocessed data

In [21]:
fhvhv_2020_df = fhvhv_2020_df.reset_index(drop=True)
fhvhv_2020_df.to_feather(r'/Volumes/E/data/preprocessed_data/fhvhv_2020.feather')

fhvhv_covid_df.to_csv(r'/Volumes/E/data/preprocessed_data/fhvhv_covid.csv',index=False)